# RouteYou interfacing for COGENT

In [ ]:
ROUTEYOU_KEY = '<FILL IN>'

In [ ]:
import requests
from random import randint
import json

class RouteYou_Json_Client:
    key = None
    def __init__(self, service, version, timeout = 10, token = None):
        self.url = 'http://api.routeyou.com/%version%/json/%service%/%session%/worker'
        self.key = RouteYou_Json_Client.key
        self.token = token
        
        self._service = service
        self._version = version
        self._timeout = timeout
        
        self._needsToken = True
        self._httpClient = None
    def __getattr__(self, name):
        #magic function to call methods in RouteYou webservice
        def wrapper(*args, **kwargs):
            return self._execurteCall(name, args)
        return wrapper
    def _getHttpClient(self):
        self.url = self.url.replace('%service%',self._service)
        self.url = self.url.replace('%version%',self._version)
        if self.token is None:
            self.url = self.url.replace('%session%','k-' + self.key)
        else:
            self.url = self.url.replace('%session%',self.token)
    def _execurteCall(self, method, params, retry = False):
        self._getHttpClient()
        id = randint(1,1000) 
        data = {
            'jsonrpc' : self._version,
            'id' : id,
            'method' : method,
            'params' : params
            }     

        req = requests.post(self.url, json.dumps(data))
        response = req.json()
        if req.status_code == 200:
            return response
        else:
            print (req.status_code)
            return False
    def getKey(self):
        return self.key

RouteYou_Json_Client.key = ROUTEYOU_KEY

## POIs

https://help.routeyou.com/en/topic/view/622/poi-web-service-2-0 

Main problem, we cannot create POIs ourselves.

In [ ]:
import math

# Automatically calculate a bounding box based on lat, lon and a radius
class BoundingBox(object):
    def __init__(self, *args, **kwargs):
        self.lat_min = None
        self.lon_min = None
        self.lat_max = None
        self.lon_max = None


def get_bounding_box(latitude_in_degrees, longitude_in_degrees, half_side_in_km):
    assert latitude_in_degrees >= -90.0 and latitude_in_degrees  <= 90.0
    assert longitude_in_degrees >= -180.0 and longitude_in_degrees <= 180.0

    lat = math.radians(latitude_in_degrees)
    lon = math.radians(longitude_in_degrees)

    radius  = 6371
    # Radius of the parallel at given latitude
    parallel_radius = radius*math.cos(lat)

    lat_min = lat - half_side_in_km/radius
    lat_max = lat + half_side_in_km/radius
    lon_min = lon - half_side_in_km/parallel_radius
    lon_max = lon + half_side_in_km/parallel_radius
    rad2deg = math.degrees

    box = BoundingBox()
    box.lat_min = rad2deg(lat_min)
    box.lon_min = rad2deg(lon_min)
    box.lat_max = rad2deg(lat_max)
    box.lon_max = rad2deg(lon_max)

    return (box)

In [ ]:
# Regio van de oude kwaremont
heli_location_lat = 50.7767526525832
heli_location_lon = 3.521515819672684

heli_bbox = get_bounding_box(heli_location_lat, heli_location_lon, 2)
ghent_bbox = BoundingBox()
ghent_bbox.lat_min = 51.053083
ghent_bbox.lon_min = 3.711850
ghent_bbox.lat_max = 51.055882
ghent_bbox.lon_max = 3.719387

In [ ]:
#example call to route webservice 2.0
# Center of Ghent: 3.721114,51.052071,3.757249,51.060831
poiService = RouteYou_Json_Client('poi', '2.0')
print(poiService.search({'bounds': {'min': {'lat': heli_bbox.lat_min, 'lon': heli_bbox.lon_min}, 'max': {'lat': heli_bbox.lat_max, 'lon': heli_bbox.lon_max}}})['result'])

[{'id': 1602181, 'location': {'id': 47503560}, 'type': {'id': 239, 'name': {'es': 'Pendiente', 'fr': 'Chantier', 'ca': 'Pendent', 'nl': 'Helling', 'de': 'Steigung', 'it': None, 'en': 'Slope'}, 'languages': ['es', 'fr', 'ca', 'nl', 'de', 'it', 'en'], 'category': {'id': 2}}, 'owner': {'id': 165089, 'flag': None, 'avatarRevision': 1, 'logoRevision': 0, 'headerRevision': 0, 'name': {'nl': 'Wikipedia'}, 'languages': ['nl'], 'permission': {'writable': False}}, 'permission': {'id': 0, 'read': 'public', 'write': 'group', 'writable': False}, 'centroid': {'wkt': 'POINT(3.529 50.769)'}, 'bounds': {'wkt': 'BOX(3.529 50.769,3.529 50.769)'}, 'flag': 'be', 'score': 0.794, 'commercialScore': 0, 'configuration': None, 'media': [{'owner': {'id': 165089}, 'score': 0.752753, 'copyright': {'id': 6}, 'width': {'de': 450, 'en': 450, 'it': 450, 'fr': 450, 'es': 450, 'ca': 450, 'nl': 450}, 'infoUri': {'de': None, 'en': None, 'it': None, 'fr': None, 'es': None, 'ca': None, 'nl': None}, 'id': 812658, 'source': {

Let's start with a bounding box of Ghent ( http://bboxfinder.com/#51.029298,3.687458,51.066335,3.753204)
  3.687458,51.029298,3.753204,51.066335

In [ ]:
gent_lat = (51.029298, 61.066335)
gent_lon = (3.687458, 4.753204)

poiService = RouteYou_Json_Client('poi', '2.0')
poi_collection = poiService.search({
    "bounds": {
        "min": {
          "lat": gent_lat[0],
          "lon": gent_lon[0]
        }, 
        "max": {
          "lat": gent_lat[1],
          "lon": gent_lon[1]
        }
    },
    "limit": 50,
    "offset": 100 
})

In [ ]:
def printkeys(json):
  for k in json:
    print(k)
  return

def printPOI(result):
  print(
    result['type']['name']['nl']
  )
  print(
    result['centroid']['wkt']
  )
  print(
    result['name']['nl']
  )

  return

### Manual exploration of the results

In [ ]:
pois = poi_collection['result']
print(len(pois))
#waarom maar 10?

for p in pois:
  printPOI(p)
  print()



10
Begraafplaats
POINT(4.3682325241382 51.165612279607)
Domein Schoonselhof

Historische plaats
POINT(3.72590278 51.04258611)
Sint-Pietersplein

Toren
POINT(3.72583333 51.04472222)
Boekentoren

Stadhuis
POINT(3.725447 51.054375)
Stadhuis

Kathedraal
POINT(4.40152750635985 51.2203551655474)
Onze-Lieve-Vrouwe-Kathedraal

Winkelstraat
POINT(4.4091154 51.2180888)
Meir

Treinstation
POINT(4.4207019 51.2169691)
Station Antwerpen-Centraal

Plein
POINT(3.723979 51.0501559)
Kouter

Museum
POINT(4.14480149745941 51.1673897834844)
Mercatormuseum

Stadhuis
POINT(4.39915825396729 51.2214601529109)
Grote Markt en stadhuis



# Routing

https://help.routeyou.com/en/topic/view/430/routing-web-service-2-0#route

Creating a route, main problem we can only provide maximum 2 via-points.

Available routingids

    15: recreational cycling - nicest
    28: race cycling - nicest
    29: recreational cycling - shortest
    30: race cycling - shortest
    31: hiking - nicest
    32: race cycling - shortest (avoid cobblestones)
    33: race cycling - nicest (avoid cobblestones)
    34: inline skating - nicest
    35: inline skating - shortest
    36: gravel bike - nicest (winter)
    37: gravel bike - shortest (winter)
    38: node-to-node cycling
    39: gravel bike - nicest (summer)
    40: gravel bike - shortest (summer)
    47: MTB Sport Vlaanderen - shortest (all directions)
    48: MTB Sport Vlaanderen - shortest
    49: motor - nicest
    50: motor - shortest
    51: motor - fastest
    53: mountain bike - nicest
    54: node-to-node cycling (only paved)
    55: horse - node network
    63: hiking - node network
    76: recreational cycling - nicest (avoid unpaved)
    80: shortest - OSM (all ways)
    94: race cycling - nicest (with car)
    95: race cycling - shortest (with car)
    96: recreational cycling - shortest (only paved)
    97: mountain bike - shortest
    98: hiking - shortest
    99: node-to-node cycling (avoid slopes)


In [ ]:
# Route from AA-tower to Gravensteen
# 51.013135455152295, 3.709584074567241 --> 51.05752102565761, 3.7209650134626404
routeService = RouteYou_Json_Client('routing', '2.0')
print(routeService.route([{'lon': 3.709584074567241, 'lat': 51.013135455152295},{'lon': 3.7209650134626404, 'lat': 51.05752102565761}], 31, {'add': {'elevation': True, 'instructions': True}, 'newInstructionFormat': True}))

{'result': {'name': {'en': 'No title'}, 'nameGenerated': {'en': True}, 'subName': {'en': None}, 'description': {'en': 'Routing: Hiking - nicest'}, 'descriptionGenerated': {'en': True}, 'begin': {'wkt': 'POINT(3.70958 51.01312)'}, 'end': {'wkt': 'POINT(3.72113 51.05755)'}, 'length': 7076.8765314185375, 'bounds': {'wkt': 'BOX(3.70436 51.01132,3.72341 51.05755)'}, 'score': 0, 'beginAddress': None, 'endAddress': None, 'languages': ['en'], 'type': {'id': 0, 'name': {'en': 'All routes'}}, 'media': [], 'infoUri': {'en': None}, 'duration': 580927, 'geometry': {'google': '_ojvH{osU?p@X?Ff@T?V@V~@Pb@b@x@j@x@\\Z~BbBOXYz@C`@?xAA~EHvBEVOPCJc@]{AqAOM[GKK}BuEq@eAgCgCs@_@WYeG_Dk@]q@_@uFsC_@CyAa@}@I{@?UBA?[Cy@DQJg@@mEX{ABqBES?Uu@MCSs@mCmIgAkDwCeJyB_HoFuP[eA}A_FM_@MEKYIDCGIWESsBsGK_@cCO_BW}CSg@Iq@SqEYi@KiAg@cDgB{BtPgBAH]Jo@QYkBmIyFfNETO~@?\\@NGBI?EfAOMO@[Tm@r@{@BYE]QKOEH[P]XJ`A@Rm@JI@o@HOJi@j@I[m@FMHQTIDKEUSUI[Ca@Cg@s@{@u@EF]DENYWI\\yCcCkBfIQh@KCqBd@_@@{BmAq@cAQ`@GUGKMGs@`BkBaIUq@kB{DOQe@[eAe@s@?iENuBRB

## Google Places API

https://github.com/googlemaps/google-maps-services-python

We need an API key to use them!

In [ ]:
! pip install -U googlemaps

## Overpass queries

Overview of different amenities
https://wiki.openstreetmap.org/wiki/Key:amenity 

Cultural heritage related ones https://wiki.openstreetmap.org/wiki/Key:amenity#Entertainment,_Arts_&_Culture

In [ ]:
!pip install overpy
!pip install geojson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.9/47.9 KB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for overpy: filename=overpy-0.6-py3-none-any.whl size=14107 sha256=5e0864d9290a86ce34902604c3e964755e240d3b6b830d31e46183c9675d40cf
  Stored in directory: /root/.cache/pip/wheels/69/6f/9b/c4e56235399fbce16dfcd71a65bd2af0e5882684cc4fa2ec70
Successfully built overpy
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
GHENT_BBOX = (51.029,3.67, 51.067,3.76)
query = f"""
            (
            node
                ['amenity']
                {GHENT_BBOX};
            node
                ['shop']
                {GHENT_BBOX};
            way
                ['shop']
                {GHENT_BBOX};
            way
                ['amenity']
                {GHENT_BBOX};
            relation
                ['amenity']
                {GHENT_BBOX};
            relation
                ['shop']
                {GHENT_BBOX};
            );
            out geom;
"""

park_query = f"""
            (
                node
                    ['leisure'='park']
                    {GHENT_BBOX};
                way
                    ['leisure'='park']
                    {GHENT_BBOX};
                relation
                    ['leisure'='park']
                    {GHENT_BBOX};
            );
            out geom;
"""

tourism_query = f"""
            (
                node
                    ['tourism']
                    {GHENT_BBOX};
                
            );
            out geom;
"""

In [ ]:
import overpy
import geojson

api = overpy.Overpass()

# fetch all ways and nodes
#result = api.query(park_query)
result = api.query(tourism_query)
output = []

for node in result.nodes:
    print(node.tags)
    print(f"{node.tags['name'] if 'name' in node.tags else None} ({node.tags['amenity'] if 'amenity' in node.tags else None})")
    print(f"\t Lat: {node.lat}, Lon: {node.lon}")

    output_line = node.tags
    output_line['coordinate'] = {'latitude': float(node.lat), 'longitude': float(node.lon)}

    output.append(output_line)
    print()

{'addr:housenumber': '43', 'addr:postcode': '9000', 'addr:street': 'Jozef Guislainstraat', 'email': 'info@museumdrguislain.be', 'fee': 'yes', 'name': 'Museum Dr. Guislain', 'opening_hours': 'Tu-Fr 09:00-17:00; Sa-Su 13:00-17:00', 'phone': '+32 9 398 69 50', 'toilets:wheelchair': 'yes', 'tourism': 'museum', 'website': 'http://www.museumdrguislain.be', 'wheelchair': 'yes', 'wikidata': 'Q2934898', 'wikimedia_commons': 'Category:Museum_Dr_Guislain'}
Museum Dr. Guislain (None)
	 Lat: 51.0668514, Lon: 3.7033213

{'tourism': 'artwork', 'wikidata': 'Q107482658'}
None (None)
	 Lat: 51.0313179, Lon: 3.7195611

{'artwork_type': 'statue', 'name': 'Ros Beiaard', 'subject:wikidata': 'Q658782', 'tourism': 'artwork', 'wikidata': 'Q88889022'}
Ros Beiaard (None)
	 Lat: 51.0317241, Lon: 3.7177118

{'brand': 'Ibis', 'brand:wikidata': 'Q920166', 'name': 'Ibis Gent Centrum Opera', 'operator': 'Ibis', 'tourism': 'hotel'}
Ibis Gent Centrum Opera (None)
	 Lat: 51.0484729, Lon: 3.7211612

{'name': 'Ghent River 

In [ ]:
print(len(output))
print(output[0])

362
{'addr:housenumber': '43', 'addr:postcode': '9000', 'addr:street': 'Jozef Guislainstraat', 'email': 'info@museumdrguislain.be', 'fee': 'yes', 'name': 'Museum Dr. Guislain', 'opening_hours': 'Tu-Fr 09:00-17:00; Sa-Su 13:00-17:00', 'phone': '+32 9 398 69 50', 'toilets:wheelchair': 'yes', 'tourism': 'museum', 'website': 'http://www.museumdrguislain.be', 'wheelchair': 'yes', 'wikidata': 'Q2934898', 'wikimedia_commons': 'Category:Museum_Dr_Guislain', 'coordinate': {'latitude': 51.0668514, 'longitude': 3.7033213}}


In [ ]:
for rel in result.relations:
    tags = rel.tags
    print(rel.members)
    print(f"{tags['name'] if 'name' in tags else None} ({tags['amenity'] if 'amenity' in tags else None})")
    print(f"\t Lat: {rel.center_lat}, Lon: {rel.center_lon}")
    print()

    #output_line = rel.tags
    #output_line['coordinate'] = {'latitude': rel.center_lat, 'longitude': rel.center_lat}

    #output.append(output_line)
    

[<overpy.RelationWay ref=1058302460 role=None>, <overpy.RelationWay ref=52199456 role=None>, <overpy.RelationWay ref=440889549 role=None>, <overpy.RelationWay ref=700006491 role=None>, <overpy.RelationWay ref=111695946 role=None>, <overpy.RelationWay ref=700006490 role=None>]
Overpoortstraat (None)
	 Lat: None, Lon: None



In [ ]:
for way in result.ways:
    tags = way.tags
    if 'amenity' in tags and tags['amenity'] in ['arts_centre', 'exhibition_centre', 'cinema', 'events_venue','library','toy_library','music_school','school','university','fountain','music_venue']:
        print(f"{tags['name'] if 'name' in tags else None} ({tags['amenity'] if 'amenity' in tags else None})")
        print(f"\t Lat: {nodes[0].lat}, Lon: {nodes[0].lon}")
        nodes = way.get_nodes(resolve_missing=True)
    
        output_line = way.tags
        output_line['coordinate']={'latitude': float(nodes[0].lat), 'longitude': float(nodes[0].lon)}

        output.append(output_line)

In [ ]:
import json

with open('pois.json', 'w+') as fp:
    json.dump(output, fp)
